In [95]:
import json
import numpy as np
import sklearn
import sklearn.metrics
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import snap
import random
from numpy.random import choice
import numpy as np
from collections import defaultdict
# from playlist import *
import playlist
reload(playlist)

<module 'playlist' from 'playlist.pyc'>

In [96]:
playlist_list = playlist.get_playlist_list()

In [97]:
len(playlist_list)

1000

In [98]:
track_list = set([])
for i in range(len(playlist_list)):
    for j in range(playlist_list[i]['num_tracks']):
        track_uri = playlist_list[i]['tracks'][j]['track_uri']
        track_list.add(track_uri)
track_list = list(sorted(track_list))
track_order_dict = dict(zip(track_list, range(len(track_list))))

In [99]:
len(track_list)

34443

In [100]:
feature_matrix = np.zeros((len(playlist_list), len(track_list)))
feature_matrix.shape

(1000, 34443)

In [101]:
for i in range(len(playlist_list)):
    for j in range(playlist_list[i]['num_tracks']):
        track_uri = playlist_list[i]['tracks'][j]['track_uri']
        k = track_order_dict[track_uri]
        feature_matrix[i][k] = 1    

In [102]:
# if 1 in feature_matrix[:, :]:
#     print"1"

In [77]:
from sklearn.decomposition import NMF
model = NMF(n_components=100, init='random', random_state=0)
W = model.fit_transform(feature_matrix)
H = model.components_

In [78]:
model.reconstruction_err_

213.38288469030252

In [79]:
import gzip
import pickle

with gzip.open('track_H_1k_100n.pickle.gz','wb') as output:
    pickle.dump(H ,output)

In [103]:
with gzip.open("track_H_1k_30n.pickle.gz", "rb") as inp:
    H = pickle.load(inp)

In [104]:
W.shape

(1000, 100)

In [105]:
H.shape

(30, 34443)

In [106]:
(H.T).shape

(34443, 30)

In [107]:
track_H_df = pd.DataFrame(data=H.T, index = track_list)

In [108]:
track_H_df.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
spotify:track:000mA0etY38nKdvf1N04af,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
spotify:track:000xQL6tZNLJzIrtIgxqSl,0.0,0.097767,0.023119,0.0,0.033683,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
spotify:track:006AVH7fq061voGXkUiII4,0.0,0.000425,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.001429,0.0,0.0,0.0,0.000182,0.0,0.0,0.000000
spotify:track:006PJvsr6CyV3JdBf7wiNF,0.0,0.000000,0.000000,0.0,0.622206,0.0,0.000104,0.000000,0.0,0.0,...,0.000389,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
spotify:track:006yrnQMCZpiUgkR612gC8,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.019741,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.027449


In [109]:
track_H_df = playlist.normalize_dataframe(track_H_df)
track_H_df.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
spotify:track:000mA0etY38nKdvf1N04af,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
spotify:track:000xQL6tZNLJzIrtIgxqSl,0.0,0.160165,0.016014,0.0,0.013145,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
spotify:track:006AVH7fq061voGXkUiII4,0.0,0.000695,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.000453,0.0,0.0,0.0,0.000142,0.0,0.0,0.000000
spotify:track:006PJvsr6CyV3JdBf7wiNF,0.0,0.000000,0.000000,0.0,0.242820,0.0,0.000055,0.000000,0.0,0.0,...,0.000268,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
spotify:track:006yrnQMCZpiUgkR612gC8,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.006634,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.012479


In [110]:
def get_track_embedding(track_uri):
    return track_H_df.loc[track_uri].values

In [111]:
cached_playlist_embedding = {}

def get_playlist_embedding(playlist_id):
    # check if already in cache 
    if playlist_id in cached_playlist_embedding:
        return cached_playlist_embedding[playlist_id]

    embedding_list = []
    for i in range(playlist_list[playlist_id]['num_tracks']):
        track_uri = playlist_list[playlist_id]['tracks'][i]['track_uri']
    #     print(type(track_df.loc[track_uri]))
        embedding_list.append(get_track_embedding(track_uri))
    embedding = np.mean(embedding_list, axis = 0)
    # put in the cache
    cached_playlist_embedding[playlist_id] = embedding
    return embedding

In [112]:
def cal_cos_similarity(playlist_id_1, playlist_id_2):
        similarity = sklearn.metrics.pairwise.cosine_similarity(get_playlist_embedding(playlist_id_1).reshape(1, -1), 
                                                                get_playlist_embedding(playlist_id_2).reshape(1, -1))
#     similarity = sklearn.metrics.pairwise.euclidean_distances(get_playlist_embedding(playlist_id_1).reshape(1, -1), 
#                                                                 get_playlist_embedding(playlist_id_2).reshape(1, -1))
        return similarity[0][0]
#     return -similarity[0][0]

In [113]:
def biased_H_random_walk_playlist(query,g, beta = 1):
    
    N = 1000
    totSteps = 0
    nHighVisited = 0
    nt = 200
    nv = 5
    num_visits = defaultdict(int)
    
    def weight_cal(edge):
        weight = cal_cos_similarity(query,edge)
        return weight

    while totSteps < N or nHighVisited > nt: 
        currPlaylist = query
        # Number of iterations
        currSteps = 5
        for i in range(currSteps):
            # takes 1 step (from a playlist to track)
            edges = list(g.GetNI(currPlaylist).GetOutEdges())
            currTrack = random.choice(edges)
            # takes 1 step (from a track to playlist)
            edges = list(g.GetNI(currTrack).GetOutEdges())
#             weights = [f(edge) for edge in edges]
            weights = [weight_cal(edge) for edge in edges]
            weights_np = np.asarray(weights)

# uncomment to do softmax
# see https://stackoverflow.com/questions/34968722/how-to-implement-the-softmax-function-in-python
#             # this bit makes it more stable numerically
            max_weight = np.max(weights_np)
            weights_np -= max_weight
            # but does not change the answer
            weights_np = np.exp(beta * weights_np)

            prob_dis = weights_np / float(sum(weights_np))
            currPlaylist = int(choice(edges, 1, p=prob_dis))
            num_visits[currTrack] += 1
            if num_visits[currTrack] == nv:
                nHighVisited += 1
        totSteps += currSteps
    return num_visits

In [114]:
def cal_most_visited_playlist(playlist_list, playlist_id):
    dropped_track_index = random.choice(range(len(playlist_list[playlist_id])))
#         for dropped_track_index in range(len(playlist_list[playlist_id])):
    g, dropped_track_id = playlist.make_graph_drop_edge(playlist_list, playlist_id, dropped_track_index)
#             print('1', playlist_id, dropped_track_id)
#             tracks in the playlist graphs where one is dropped
    if len(list(g.GetNI(playlist_id).GetOutEdges())) < 5:
        return None, None
    query = playlist_id
    num_visits = biased_H_random_walk_playlist(query,g)
#     num_visits = biased_audio_random_walk_playlist(query,g)
#     num_visits = highest_visited_random_walk_playlist(query, g)
#     num_visits =biased_modified_at_random_walk_playlist(query, g)
    suggested_ids = sorted(num_visits, key=num_visits.get, reverse=True)
    return dropped_track_id, suggested_ids

In [115]:
# all_true_ids, all_suggested_ids = cal_most_visited(playlist_list)
all_true_ids, all_suggested_ids = playlist.process_playlists(cal_most_visited_playlist, playlist_list, multiprocess=False)

In [116]:
avg_precision = playlist.cal_ave_precision(600, all_true_ids, all_suggested_ids)
print avg_precision

0.17125382263
